In [1]:
!nvidia-smi

Wed Jul  5 12:42:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pip install -q langchain transformers sentence_transformers llama-index


In [4]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [5]:

class customLLM(LLM):
    model_name = "google/flan-t5-Large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())


In [6]:

hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)


In [36]:

from llama_index import SimpleDirectoryReader

bet = SimpleDirectoryReader('/content/drive/MyDrive/data/bet_health').load_data()
tumo = SimpleDirectoryReader('/content/drive/MyDrive/data/tumo_health').load_data()


In [37]:
# set number of output tokens
num_output = 10000
# set maximum input size
max_input_size = 10000
# set maximum chunk overlap
max_chunk_overlap = 1


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


In [38]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
# train for tumo
index_tumo = GPTVectorStoreIndex.from_documents(tumo, service_context=service_context, prompt_helper=prompt_helper)
# train for bet
index_bet = GPTVectorStoreIndex.from_documents(bet, service_context=service_context, prompt_helper=prompt_helper)

In [39]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

In [40]:
# #test
# query_engine_tumo = index_tumo.as_query_engine()
# prompt = " what dental services does my insurance cover? "
# response = query_engine_tumo.query(prompt)
# print(response)

In [41]:
# for tumo
query_engine_tumo = index_tumo.as_query_engine()
# for bet
query_engine_bet = index_bet.as_query_engine()

In [42]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [43]:
pip install deep_translator

In [44]:
from deep_translator import GoogleTranslator

In [45]:
translator_am = GoogleTranslator(source='en', target='hy')
translator_en = GoogleTranslator(source='hy', target='en')

In [46]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [47]:
! pip install pytelegrambotapi aiogram


In [49]:
import telebot
from telebot import TeleBot
from telebot import types
from aiogram.types import ReplyKeyboardMarkup, ReplyKeyboardRemove, KeyboardButton

API_TOKEN = '6156036993:AAFP6oS_-bBJdLv1YDWUTto0PdCoZYaEihU'
bot = telebot.TeleBot(API_TOKEN)

selected_option = None  # Global variable to store the selected option

button_tumo = types.InlineKeyboardButton('Health_1', callback_data='Health_1')
button_bet = types.InlineKeyboardButton('Health_2', callback_data='Health_2')
keyboard = types.InlineKeyboardMarkup()
keyboard.add(button_tumo)
keyboard.add(button_bet)

@bot.message_handler(commands=['start'])
def start(message):
    global selected_option
    selected_option = None  # Reset the selected option
    bot.send_message(message.chat.id, 'Բարև, ես քեզ կարղ եմ օգնել ընտրել քեզ հարմար առողջապահական  ապահովաագրության տեսակ: Ընտրիր ցանկանանը', reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: call.data == 'Health_1')
def handle_health_1_callback(call):
    global selected_option
    selected_option = 'Health_1'
    bot.send_message(call.message.chat.id, 'Դուք ընտրել էք համար 1 ապահովագրության տեսակը, և ես պատրաստ եմ պատասխանել ձեր հարցերին')

@bot.callback_query_handler(func=lambda call: call.data == 'Health_2')
def handle_health_2_callback(call):
    global selected_option
    selected_option = 'Health_2'
    bot.send_message(call.message.chat.id, 'Դուք ընտրել էք համար 2 ապահովագրության տեսակը, և ես պատրասխանել ձեր հարցերին')

@bot.message_handler(func=lambda message: True)
def generate_response(message):
    global selected_option

    if selected_option == 'Health_1':
        input_text = translator_en.translate(message.text)

        query_engine_tumo = index_tumo.as_query_engine()
        test = "is the question related to your custom knowledge?"
        response_test = query_engine_tumo.query(input_text + test)
        if str(response_test) == 'yes':
            response = query_engine_tumo.query(input_text)
            bot.reply_to(message, translator_am.translate(str(response)))
        else:
            response = 'Կներեք, չեմ կարող կատարել այդ խնդրանքը, քանի որ տրամադրված ՏԵՂԵԿԱՏՎՈՒԹՅՈՒՆԸ չի վերաբերում առողջության ապահովագրությանը:'
            bot.reply_to(message, response)

    if selected_option == 'Health_2':
        input_text = translator_en.translate(message.text)

        query_engine_bet = index_bet.as_query_engine()
        test = "is the question related to your custom knowledge?"
        response_test = query_engine_bet.query(input_text + test)
        if str(response_test) == 'yes':
            response = query_engine_bet.query(input_text)
            bot.reply_to(message, translator_am.translate(str(response)))
        else:
            response = 'Կներեք, չեմ կարող կատարել այդ խնդրանքը, քանի որ տրամադրված ՏԵՂԵԿԱՏՎՈՒԹՅՈՒՆԸ չի վերաբերում առողջության ապահովագրությանը:'
            bot.reply_to(message, response)




In [50]:
# and here we actually run it
bot.polling()

In [ ]:
'Sorry, I’m afraid I cannot fulfill that request as the provided CONTEXT INFORMATION does not relate to health insurance'

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
# import os
# import json
# from deep_translator import GoogleTranslator

In [ ]:
#  translator = GoogleTranslator(source='auto', target='en')

In [ ]:
# print(translator.translate("Կարո՞ղ եմ մեջքիս ռենտգեն անել:"))

Can I have an X-ray of my back?


In [ ]:
# query_engine = index.as_query_engine()

# translator_to_en = GoogleTranslator(source='auto', target='en')
# prompt_hy = "Կարո՞ղ եմ մեջքիս ռենտգեն անել:"
# print(prompt_hy)
# prompt_en = translator_to_en.translate(prompt_hy)
# print(prompt_en)

# response_en = query_engine.query(prompt_en)
# print(response_en)
# translator_to_hy = GoogleTranslator(source='auto', target='hy')
# response_hy = translator_to_hy.translate(str(response_en))
# print(response_hy)

Կարո՞ղ եմ մեջքիս ռենտգեն անել:
Can I have an X-ray of my back?
yes
այո
